In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import matplotlib.pyplot as plt
import json
import os

# Dataset paths
train_path = r"C:\Users\thula\Documents\Mini_Project\Dataset\training_set"
test_path = r"C:\Users\thula\Documents\Mini_Project\Dataset\test_set"

# Image parameters
IMG_SIZE = 64
BATCH_SIZE = 32
NUM_CLASSES = 26

# Data generators
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=False,
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(rescale=1./255)

train_set = train_datagen.flow_from_directory(
    train_path,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

test_set = test_datagen.flow_from_directory(
    test_path,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

# Save class indices
with open("class_indices_vgg16.json", "w") as f:
    json.dump(train_set.class_indices, f)

# Load VGG16 base model
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))

# Freeze base model layers
for layer in base_model.layers:
    layer.trainable = False

# Add custom classifier on top
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(NUM_CLASSES, activation='softmax')(x)

# Build model
model = Model(inputs=base_model.input, outputs=predictions)

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Callbacks
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-7)

# Train model
EPOCHS = 10
STEPS_PER_EPOCH = 120
VALIDATION_STEPS = test_set.samples // BATCH_SIZE

history = model.fit(
    train_set,
    steps_per_epoch=STEPS_PER_EPOCH,
    validation_data=test_set,
    validation_steps=VALIDATION_STEPS,
    epochs=EPOCHS,
    callbacks=[early_stop, reduce_lr]
)

# Save model
model.save("asl_vgg16_model.h5")

# Plot accuracy and loss


# Evaluate on test set
loss, accuracy = model.evaluate(test_set)
print(f"\n✅ Test Accuracy (VGG16): {accuracy*100:.2f}%")


In [2]:
loss, accuracy = model.evaluate(test_set)
print(f"\n✅ Test Accuracy (VGG16): {accuracy*100:.2f}%")

1/1 [==============================] - 1s 595ms/step - loss: 2.1132 - accuracy: 0.6154

✅ Test Accuracy (VGG16): 61.54%
